In [2]:
//sc라는 인스턴스를 통해 스파크 컨턱스트가 sc를 사용할 수 있도록 한다.
//sc에는 다양한 메소드가 존재한다. sc에서 사용할 많은 부분은 RDD를 만드는 것이다.
//스파크에서는 빅데이터를 멀티플 노드에 분산되어 데이터를 분석하기 떄문에 
//데이터 셋을 하나의 노드에서 사용하지 않고 여러개에서 나누어서 사용하게 된다.
//나누어진 데이터셋을 RDD라는 데이터 구조를 통해서 사용한다.
//sc로 데이터를 불러오거나 하면 다 RDD로 만들어 지게 된다.
sc

res0: org.apache.spark.SparkContext = org.apache.spark.SparkContext@76fdf710


In [3]:
val rdd = sc.parallelize(Array(1,2,2,4), 4)
//RDD를 만드는 Parallelize 메소드를 이용한 방법

rdd: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[0] at parallelize at <console>:25


In [4]:
val rawblocks = sc.textFile("Data/linkage")
//RDD를 만드는 로컬 데이터셋을 불러오는 방법

rawblocks: org.apache.spark.rdd.RDD[String] = Data/linkage MapPartitionsRDD[2] at textFile at <console>:25


In [5]:
rawblocks.first
//RDD의 first라는 메소드는 첫번째 로우를 출력해주는 메소드 이다.
//확인해보니 데이터가 처음부터 등장하는 것이 아닌 데이터셋에 헤더가 존재하는 구나를
//알 수 있다. 즉 칼럼의 이름을 먼저 보고 들어갈 수 있다는 것이다.

res1: String = "id_1","id_2","cmp_fname_c1","cmp_fname_c2","cmp_lname_c1","cmp_lname_c2","cmp_sex","cmp_bd","cmp_bm","cmp_by","cmp_plz","is_match"


In [6]:
val head = rawblocks.take(10)
//RDD의 take라는 메소드를 통해서는 몇개의 로우를 가져와서 볼 수 있다.

head: Array[String] = Array("id_1","id_2","cmp_fname_c1","cmp_fname_c2","cmp_lname_c1","cmp_lname_c2","cmp_sex","cmp_bd","cmp_bm","cmp_by","cmp_plz","is_match", 37291,53113,0.833333333333333,?,1,?,1,1,1,1,0,TRUE, 39086,47614,1,?,1,?,1,1,1,1,1,TRUE, 70031,70237,1,?,1,?,1,1,1,1,1,TRUE, 84795,97439,1,?,1,?,1,1,1,1,1,TRUE, 36950,42116,1,?,1,1,1,1,1,1,1,TRUE, 42413,48491,1,?,1,?,1,1,1,1,1,TRUE, 25965,64753,1,?,1,?,1,1,1,1,1,TRUE, 49451,90407,1,?,1,?,1,1,1,1,0,TRUE, 39932,40902,1,?,1,?,1,1,1,1,1,TRUE)


In [7]:
head.foreach(println)
//데이터를 정리해서 보기 위해서 foreach(println) 을 통해서 볼 수 있다.
//매칭스코어를 볼때 각 레이블에 대해서 매칭이 1이라면 그냥 같은 사람일 수 밖에 
//없다. String의 비교 경우 비슷한 정도를 비교해 0 ~ 1 사이의 값을 넣어놓았다.

//foreach()메소드는 이 RDD의 각 로우에 안의 함수를 각각 적용시키겠다 라고
//이해할 수 있다. 그래서 하나의 함수를 메소드의 인자로 넣을 수 있는 것이다.
//즉 메소드가 메소드의 인자로 들어갈 수 있게 되는 것이다.
//foreach는 각 로우에 대해서 안의 인자로 들어온 함수를 적용시키겠다는 뜻이다.

"id_1","id_2","cmp_fname_c1","cmp_fname_c2","cmp_lname_c1","cmp_lname_c2","cmp_sex","cmp_bd","cmp_bm","cmp_by","cmp_plz","is_match"
37291,53113,0.833333333333333,?,1,?,1,1,1,1,0,TRUE
39086,47614,1,?,1,?,1,1,1,1,1,TRUE
70031,70237,1,?,1,?,1,1,1,1,1,TRUE
84795,97439,1,?,1,?,1,1,1,1,1,TRUE
36950,42116,1,?,1,1,1,1,1,1,1,TRUE
42413,48491,1,?,1,?,1,1,1,1,1,TRUE
25965,64753,1,?,1,?,1,1,1,1,1,TRUE
49451,90407,1,?,1,?,1,1,1,1,0,TRUE
39932,40902,1,?,1,?,1,1,1,1,1,TRUE


In [ ]:
def isHeader(line: String) = line.contains("id_1")
//해당되는 헤더 즉 id_1이 포함된 로우는 첫쨰 로우니까 그걸 가져온다는 것이다.

In [15]:
//filter메소드를 사용해 안의 메소드의 값이 참이면 그 부분만 출력하겠다는
//필터링 메소드 이다.
head.filter(isHeader).foreach(println)

<console>: 30: error: not found: value isHeader

In [ ]:
val noheader = rawblocks.filter(x => !isHeader(x))
//noheader라는 함수를 만들어서 isheader가 false인 것만 필터링 한 것이다.
noheader.first

In [18]:
//방금까지는 RDD만을 가지고 보았었다. 새로운 데이터 Abstraction 방식이 
//있는데 dataFrame이라는 방식이다. RDD와 다르게 데이터베이스에 좀 더
//가까운 형태이다. 즉 스파크에서 제공하는 어떤 테이블로 생각할 수 있다.
//이 데이터 프레임 역시 RDD와 같게 쪼갤 수 있다.
//sc가 아닌 spark라는 오브젝트를 사용하게 되고 이는 스파크 세션을 이용하는 
//오브젝트 이다.
val prev = spark.read.csv("Data/linkage")

//데이터를 읽어오는 방법은 spark.read.scv 라는 형태로 불러오고 그걸 변수에 넣는다.
//csv를 read하는 메소드 이고 다양한 파일에 대해 read 메소드가 있을 것이다.

prev: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 10 more fields]


In [19]:
prev.show()
//만들어진 데이터 프레임에 show() 메소드를 적용하면 이처럼 테이블과 같은 느낌으로 나오게 된다.
//그런데 우리가 read.csv 시에 옵션을 언주었더니 헤더가 테이블에 포함되버렸다.
//즉 칼럼 이름으로 만들기 위한 옵션을 사용할 수 있다.

+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
|  _c0|  _c1|         _c2|         _c3|         _c4|         _c5|    _c6|   _c7|   _c8|   _c9|   _c10|    _c11|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
| 3148| 8326|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|14055|94934|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|33948|34740|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|  946|71870|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|64880|71676|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|  

In [1]:
//헤더옵션 헤더가 있으면 true / 그리고 nullvalue는 뭐로 표현되있는지 여기선 ? 이다.
//그리고 infretSchema 는 스키마를 인퍼링 하려면 true 이다.
val parsed = spark.read.option("header", "true").option("nullValue","?").option("inferScehma","true").csv("Data/linkage")

Intitializing Scala interpreter ...

Spark Web UI available at http://localhost:4040
SparkContext available as 'sc' (version = 2.4.6, master = local[*], app id = local-1631955373148)
SparkSession available as 'spark'


parsed: org.apache.spark.sql.DataFrame = [id_1: string, id_2: string ... 10 more fields]


In [ ]:
//우리가 스키마 인퍼링을 true로 두었기 떄문에 스키마를 인퍼링 할 수 있다.
parsed.printSchema()
parsed.cache() // 캐시에 올리는 것도 중요한 듯 하다 오버로딩이 많이 준다.

//보면 헤더가 true여서 각 칼럼의 데이터 타입과 이름을 볼 수 있다.
//데이터 프레임을 이용하면 RDD보다 데이터 구조를 더 잘 볼 수 있다.

In [30]:
parsed.show()
//이처럼 파싱을 하고 show 하면 위쪽과는 다르게 나오는 것을 볼 수 있다.

+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| 3148| 8326|           1|        null|           1|        null|      1|     1|     1|     1|      1|    TRUE|
|14055|94934|           1|        null|           1|        null|      1|     1|     1|     1|      1|    TRUE|
|33948|34740|           1|        null|           1|        null|      1|     1|     1|     1|      1|    TRUE|
|  946|71870|           1|        null|           1|        null|      1|     1|     1|     1|      1|    TRUE|
|64880|71676|           1|        null|           1|        null|      1|     1|     1|     1|      1|    TRUE|
|25739|45991|           1|        null|           1|        null|      1|     1|     1|     1|      1|  

In [31]:
//데이터 프레임에는 count 명령어도 있다.
//사용하면 전체 데이터 셋이 몇갠지 알 수 있디.
parsed.count()

res10: Long = 5749132


In [32]:
//실제로 파싱된 데이터를 계속 사용할 수도 있기 떄문에
//메모리 캐시에 캐싱하는 명령어 parsed.cache() 도 있다.

//또한 그룹핑을 해서 확인하는 메소드도 있다.
//is_match가 true냐 아니냐가 중요한 것이라 이제 명령어를 분해해보면
//그룹바이를 해라 뭐로? is_match의 트루 펄스를 count 해서 그룹바이 해라
//그래서 그 결과를 show() 해라 라는 뜻이다.
parsed.groupBy("is_match").count().show()

+--------+-------+
|is_match|  count|
+--------+-------+
|   FALSE|5728201|
|    TRUE|  20931|
+--------+-------+



In [ ]:
val summary = parsed.describe()
//describe() 를 통해 평균 , mean맥스 등등 다양한 개요를 볼 수 있다.

In [ ]:
summary.show()

In [ ]:
//summary.select("요소","요소") 를 통해 요소들만을 골라서 볼 수도 있다.

In [ ]:
val matches = parsed.where("is_match = true")
val matchsum = matches.decribe()

In [ ]:
matchsum.show()